In [1]:
import pandas as pd
from indra.databases import hgnc_client
from protmapper import ProtMapper
from protmapper import uniprot_client
from requests.exceptions import HTTPError
import csv

In [2]:
df = pd.read_excel('data/MCF10A EGF TC LargevMini Comp 10_16.xlsx', index_col=0)

In [3]:
values_cols = ['default_126_sn_sum', 'default_127n_sn_sum', 'default_127c_sn_sum', 'default_128n_sn_sum',
               'default_128c_sn_sum', 'default_129n_sn_sum', 'default_129c_sn_sum', 'default_130n_sn_sum',
               'default_130c_sn_sum', 'default_131_sn_sum', 'default_131c_sn_sum',]
site_cols = ['Protein Id', 'gene_symbol', 'Site Position', 'Motif']

In [4]:
def normalize_gene_symbol(id_str, gene_name):
    # First try to get the HGNC ID from the given gene symbol
    hgnc_id = hgnc_client.get_current_hgnc_id(gene_name)
    # If None, that means the given symbol is not a current or previous gene symbol;
    # If a list, there are multiple current gene symbols for the gene symbol
    if hgnc_id is None or isinstance(hgnc_id, list):
        # Get the Uniprot ID from the ID str
        id_str_up_id = id_str.split('|')[1].split('-')[0]
        # Get the gene name from the Uniprot ID
        up_gene_name = uniprot_client.get_gene_name(id_str_up_id)
        # Get the HGNC ID from the Uniprot gene name
        hgnc_id = hgnc_client.get_hgnc_id(up_gene_name)
    # Get the canonical Uniprot ID given the HGNC ID
    up_id = hgnc_client.get_uniprot_id(hgnc_id)
    # If success, get the normalized gene symbol
    if up_id:
        norm_sym = hgnc_client.get_hgnc_name(hgnc_id)
        return norm_sym
    else:
        return None

In [5]:
def normalize_motif(motif): 
    norm_s = motif.replace('x', '')
    offset = motif.find(norm_s)
    if offset == 0:
        pos = 7
    else:
        pos = 7 - offset
    return (norm_s, pos)

In [6]:
ok = []
ms_none = []
ms_diff = []
http_err = []
pm = ProtMapper()
cols = site_cols + values_cols
for row in df[cols].itertuples():
    site_id, id_str, gene, pos, motif = row[0:5]
    values = row[5:]
    # TODO: Currently skips peptides with multiple phosphorylations
    if ';' in str(site_id):
        continue
    norm_sym = normalize_gene_symbol(id_str, gene)
    norm_motif, norm_pos = normalize_motif(motif)
    if norm_sym is None:
        continue
    try:
        ms = pm.map_peptide_to_human_ref(norm_sym, 'hgnc', norm_motif, norm_pos)
    except HTTPError as err:
        http_err.append((site_id, id_str, gene, pos, motif))
        continue
    if ms.mapped_pos is None:
        ms_none.append((site_id, id_str, gene, pos, motif))
        continue
    elif int(ms.mapped_pos) != int(pos):        
        ms_diff.append((site_id, id_str, gene, pos, motif))
        ok.append((site_id, id_str, gene, ms.mapped_pos, motif) + tuple(values))
    else:
        ok.append((site_id, id_str, gene, pos, motif) + tuple(values))

INFO: [2019-09-24 14:05:38] protmapper.uniprot_client - Loading Swissprot sequences...
INFO: [2019-09-24 14:05:41] protmapper.uniprot_client - Loading Uniprot isoform sequences...


In [10]:
header = ['idcol', 'symcol', 'sitescol', 'effectcol'] + values_cols
csv_rows = [header]
for site_row in ok:
    site_id, id_str, gene, pos, motif = site_row[0:5]
    values = list(site_row[5:])
    norm_id_str = id_str.replace('|', '-')
    csv_row = ['%s-%s-%s-%s' % (site_id, norm_id_str, gene, pos),
           gene, '%s%d' % (motif[6], int(pos)), ''] + values          
    csv_rows.append(csv_row)
with open('phosphosites.tsv', 'wt') as f:
    csvwriter = csv.writer(f, delimiter='\t')
    csvwriter.writerows(csv_rows)
